# Let's build all kind of auto-de models.

In [5]:
#imports
import os
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from ioh_xplainer import explainer
import pandas as pd
import ioh
from scipy.stats import qmc
from ConfigSpace import ConfigurationSpace
from ConfigSpace.util import generate_grid
from IPython.display import display
from time import monotonic
from sklearn.tree import DecisionTreeClassifier 
from sklearn import tree
from matplotlib import pyplot as plt

In [6]:
ela_df = pd.read_pickle("ela-features.pkl")
features= ['mutation_base', 'mutation_reference',
       'mutation_n_comps', 'use_archive', 'crossover', 'adaptation_method',
       'lpsr']
ela_df.describe()

,mutation_n_comps,dim,fid,iid,auc,ela_meta.lin_simple.adj_r2,ela_meta.lin_simple.intercept,ela_meta.lin_simple.coef.min,ela_meta.lin_simple.coef.max,ela_meta.lin_simple.coef.max_by_min,...,disp.diff_median_05,disp.diff_median_10,disp.diff_median_25,disp.costs_runtime,ic.h_max,ic.eps_s,ic.eps_max,ic.eps_ratio,ic.m0,ic.costs_runtime
count,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,...,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000,238.000000
mean,1.420168,17.500000,12.529412,2.983193,6271.617906,0.415939,0.306031,0.001585,0.020391,208.855841,...,-2.226518,-1.822927,-1.228216,0.066090,0.814488,-1.163979,0.008995,-1.970500,0.570751,17.531552
std,0.494626,12.526344,6.958374,1.411126,2217.138544,0.309497,0.213423,0.002628,0.017609,602.809588,...,1.195817,0.981489,0.646552,0.020165,0.047347,0.394779,0.010292,0.461801,0.061397,15.829076
min,1.000000,5.000000,1.000000,1.000000,2308.641975,-0.012387,0.006332,0.000001,0.001866,1.180272,...,-4.332638,-3.542501,-2.382481,0.039957,0.661395,-1.956957,0.000030,-4.139139,0.445892,1.316971
25%,1.000000,5.000000,6.250000,2.000000,4386.882716,0.074840,0.147449,0.000103,0.006671,10.000000,...,-3.239734,-2.694112,-1.823422,0.050578,0.781022,-1.516517,0.003488,-2.137137,0.506263,1.839350
50%,1.000000,17.500000,13.000000,3.000000,5546.604938,0.445528,0.275287,0.000402,0.014511,38.558495,...,-2.250866,-1.779900,-1.182159,0.060897,0.819140,-1.316316,0.005281,-2.017017,0.585170,16.444365
75%,2.000000,30.000000,18.750000,4.000000,8297.530864,0.667934,0.395027,0.001933,0.029578,120.840372,...,-1.464608,-1.194742,-0.824243,0.079777,0.856508,-0.815816,0.011563,-1.656657,0.613727,32.137450
max,2.000000,30.000000,24.000000,5.000000,10000.000000,1.000000,0.972162,0.013651,0.102430,4335.993230,...,0.487208,0.316937,0.231666,0.146778,0.888130,-0.255255,0.066654,-0.995996,0.710421,40.080469


In [35]:
def predToNames(y):
    #transform a numerical representation back to the categorical names
    pass

#preprocess the data to train and test
for dim in ela_df['dim'].unique():
    for fid in ela_df['fid'].unique():
        print(dim, fid)
        df = ela_df[ela_df['dim'] == dim]
        X_train = df[df['fid'] != fid].copy().drop(['fid','iid','auc',*features], axis=1)
        y_train = df[df['fid'] != fid][features]
        X_test = df[df['fid'] == fid].copy().drop(['fid','iid','auc',*features], axis=1)
        y_test = df[df['fid'] == fid][features]

        X_train.replace([np.inf, -np.inf], 0, inplace=True)
        X_test.replace([np.inf, -np.inf], 0, inplace=True)
        X_train.fillna(0, inplace=True)
        X_test.fillna(0, inplace=True)

        y_train[features] = y_train[features].apply(lambda col:pd.Categorical(col).codes)
        y_test[features] = y_test[features].apply(lambda col:pd.Categorical(col).codes)
    
        dtree = DecisionTreeClassifier(max_depth=5)
        dtree.fit(X_train, y_train)
        y_pred = dtree.predict(X_test)
        #parse y_pred back
        print(dtree.predict(X_test))

5 1
   dim  ela_meta.lin_simple.adj_r2  ela_meta.lin_simple.intercept  \
0    5                    0.607936                       0.354118   
1    5                    0.832419                       0.374473   
2    5                    0.860210                       0.382606   
3    5                    0.808662                       0.322852   
4    5                    0.791429                       0.334876   

   ela_meta.lin_simple.coef.min  ela_meta.lin_simple.coef.max  \
0                      0.003697                      0.036069   
1                      0.002071                      0.036605   
2                      0.006325                      0.032233   
3                      0.002141                      0.036196   
4                      0.006205                      0.035048   

   ela_meta.lin_simple.coef.max_by_min  ela_meta.lin_w_interact.adj_r2  \
0                             9.757575                        0.607260   
1                            17.675447    